In [1]:
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle, ModelHandle
from rail.core.stage import RailStage

/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [3]:
model_file = "./test_informer.pkl"
model_handle = DS.add_data("model", data=None, handle_class=ModelHandle, path=model_file)
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

cfgfile = "./configs/solo/solo_swin_DC2.py"

In [4]:
deep_estimation_dict = dict(
    chunk_size=5,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    model=model_file,
    **deep_estimation_dict,
)

In [5]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Caching data
Processing chunk (start:end) - (0:5)
In launch function
World size: 2
Start processes
dist_url: tcp://127.0.0.1:54854
Local rank: 1
Matching objects
[01/09 17:32:11 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/09 17:32:11 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/09 17:32:11 d2.data.common]: Serialized dataset takes 1.29 MiB
Local rank: 0
Matching objects
[01/09 17:32:11 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/09 17:32:11 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/09 17:32:11 d2.data.common]: Serialized dataset takes 1.29 MiB


/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/drewoldag/code/detectron2/detectron2/engine/launch.py", line 129, in _distributed_worker
    main_func(*args)
  File "/home/drewoldag/code/rail_deepdisc/src/rail/estimation/algos/deepdisc.py", line 384, in _do_inference
    true_zs, pdfs, ids = run_batched_match_redshift(loader, predictor, ids=True)
  File "/home/drewoldag/code/deepdisc/src/deepdisc/inference/match_objects.py", line 280, in run_batched_match_redshift
    matched_gts, matched_dts = get_matched_object_inds(d, outputs)
  File "/home/drewoldag/code/deepdisc/src/deepdisc/inference/match_objects.py", line 30, in get_matched_object_inds
    gt_boxes = np.array([a["bbox"] for a in dataset_dict["annotations"]])
KeyError: 'annotations'


In [ ]:
res_ens = results_from_chunks.read()
res_ens.npdf

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
x = np.linspace(0,3,100)
ax.plot(x, res_ens[0].pdf(x)[0])
ax.plot(x, res_ens[1].pdf(x)[0])